In [1]:
import os
import glob
import numpy as np 
import pandas as pd 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 5000)

ModuleNotFoundError: No module named 'plotly'

In [ ]:
def bar_chart_multiple_choice(response_counts,title,y_axis_title,orientation='h',num_choices=15): 
    response_counts_series = pd.Series(response_counts)
    response_counts_series.index.name = ''
    pd.DataFrame(response_counts_series).to_csv('/kaggle/working/individual_charts/data/'+title+'.csv',index=True)
    fig = px.bar(response_counts_series,
             labels={"": '',"value": y_axis_title},
             text=response_counts_series.values,
             orientation=orientation,)
    fig.update_layout(showlegend=False,
                      title={'text': title+' in 2022',
                             'y':0.95,
                             'x':0.5,},
                      xaxis_range=[0,max(response_counts_series[0:num_choices]*1.2)])
    fig.write_html('/kaggle/working/individual_charts/charts/'+title+'.html')
    fig.show()
def bar_chart_multiple_choice_multiple_selection(df,title,orientation='v'):  
    df_new = df.copy()
    counts = df_new[df_new.columns[:]].count().sort_values(ascending=True)
    percentages = counts*100/len(df_new)
    percentages.index.name = ''
    percentages.to_csv('/kaggle/working/individual_charts/data/'+title+'.csv',index=True)
    if orientation is 'v':
        fig = px.bar(percentages,text=np.round(percentages.values,1),orientation=orientation)
        fig.update_layout(
            title=title+' in 2022',
            yaxis_title='% of respondents',
            showlegend=False)
        fig.write_html('/kaggle/working/individual_charts/charts/'+title+'.html')
        fig.show()
    else:
        fig = px.bar(percentages,text=np.round(percentages.values,1),orientation=orientation)
        fig.update_layout(
            title=title+' in 2022',
            xaxis_title='% of respondents',
            showlegend=False)
        fig.write_html('/kaggle/working/individual_charts/charts/'+title+'.html')
        fig.show()    

def bar_chart_multiple_years(df,x_column,y_column,title,y_axis_title):
    df.to_csv('/kaggle/working/individual_charts/data/'+title+'.csv',index=True)
    fig = px.histogram(df, x= x_column, y = y_column,barmode='group',color='year',orientation='v',
    template='ggplot2',    
                       color_discrete_map={
        '2017': px.colors.sequential.Blues[2],
        '2018': px.colors.sequential.Blues[3],
        '2019': px.colors.sequential.Blues[4],
        '2020': px.colors.sequential.Blues[5],
        '2021': px.colors.sequential.Blues[7],
        '2022': px.colors.sequential.Blues[8]})
    fig.update_layout(title=title_for_chart,yaxis=dict(title=title_for_y_axis))
    fig.update_layout(legend=dict(yanchor="auto",y=0))
    fig.write_html('/kaggle/working/individual_charts/charts/'+title+'.html')
    fig.show()
    
def create_choropleth_map(df, column, title, max_value):
    df.to_csv('/kaggle/working/individual_charts/data/'+title+'.csv',index=True)
    fig = px.choropleth(df, 
                    locations = 'country',  
                    color = column,
                    locationmode = 'country names', 
                    color_continuous_scale = 'viridis',
                    title = title,
                    range_color = [0, max_value])
    fig.update(layout=dict(title=dict(x=0.5)))
    fig.write_html('/kaggle/working/individual_charts/charts/'+title+'.html')
    fig.show()

                       

In [ ]:
def grab_subset_of_data(original_df, question_of_interest):
    # adapted from https://www.kaggle.com/siddhantsadangi/your-country-vs-the-world-24-factors-wip
    new_df = original_df.copy()
    subset_of_columns = [col for col in new_df.columns if question_of_interest in col]
    mapper = [col.split('-')[-1].lstrip() for col in subset_of_columns]
    mapping_dict = dict(zip(subset_of_columns,mapper))
    new_df = new_df[subset_of_columns].rename(columns=mapping_dict)
    new_df.dropna(how='all', subset=mapper, inplace=True)
    return new_df

def create_dataframe_of_counts(dataframe,column,rename_index,rename_column,return_percentages=False): 
    df = dataframe[column].value_counts().reset_index() 
    if return_percentages==True:
        df[column] = (df[column]*100)/(df[column].sum())
    df = pd.DataFrame(df) 
    df = df.rename({'index':rename_index, column:rename_column}, axis='columns')
    return df

def count_then_return_percent(dataframe,x_axis_title): 
    counts = dataframe[x_axis_title].value_counts(dropna=False)
    percentages = round(counts*100/(dataframe[x_axis_title].count()),1)
    return percentages

def add_year_column_to_dataframes(df_2022,df_2021,df_2020,df_2019,df_2018,df_2017=None):
    if df_2017 is not None:
        df_2022['year'] = '2022'
        df_2021['year'] = '2021'
        df_2020['year'] = '2020'
        df_2019['year'] = '2019'
        df_2018['year'] = '2018'
        df_2017['year'] = '2017'
        return df_2017,df_2018,df_2019,df_2020,df_2021,df_2022
    else:
        df_2022['year'] = '2022'
        df_2021['year'] = '2021'
        df_2020['year'] = '2020'
        df_2019['year'] = '2019'
        df_2018['year'] = '2018'
        return df_2018,df_2019,df_2020,df_2021,df_2022

def convert_df_of_counts_to_percentages(df,df_counts): 
    df_combined_percentages = df_counts.copy().astype(int)
    df_combined_percentages[0:1] = df_combined_percentages[0:1]*100/df['year'].eq('2018').sum()
    df_combined_percentages[1:2] = df_combined_percentages[1:2]*100/df['year'].eq('2019').sum()
    df_combined_percentages[2:3] = df_combined_percentages[2:3]*100/df['year'].eq('2020').sum()
    df_combined_percentages[3:4] = df_combined_percentages[3:4]*100/df['year'].eq('2021').sum()
    df_combined_percentages[4:5] = df_combined_percentages[4:5]*100/df['year'].eq('2022').sum()
    df_combined_percentages['year'] = ['2018','2019','2020','2021','2022']
    return df_combined_percentages

In [ ]:
def combine_subset_of_data_from_multiple_years(question_of_interest,x_axis_title,include_2017=None): 
    if include_2017 is not None:
        question_of_interest = question_of_interest
        df_2022 = pd.DataFrame(count_then_return_percent(responses_df_2022,question_of_interest).sort_index())
        df_2021 = pd.DataFrame(count_then_return_percent(responses_df_2021,question_of_interest).sort_index())
        df_2020 = pd.DataFrame(count_then_return_percent(responses_df_2020,question_of_interest).sort_index())
        df_2019 = pd.DataFrame(count_then_return_percent(responses_df_2019,question_of_interest).sort_index())
        df_2018 = pd.DataFrame(count_then_return_percent(responses_df_2018,question_of_interest).sort_index())
        df_2017 = pd.DataFrame(count_then_return_percent(responses_df_2017,question_of_interest).sort_index())
                add_year_column_to_dataframes(
            df_2022,
            df_2021,
            df_2020,
            df_2019,
            df_2018,
            df_2017)

        df_combined = pd.concat(
            [df_2017,
            df_2018,
            df_2019,
            df_2020,
            df_2021,
            df_2022])

        df_combined[x_axis_title] = df_combined.index
        df_combined.columns = ['percentage','year',x_axis_title]
            return df_combined
        else:
        question_of_interest = question_of_interest
        df_2022 = pd.DataFrame(count_then_return_percent(responses_df_2022,question_of_interest).sort_index())
        df_2021 = pd.DataFrame(count_then_return_percent(responses_df_2021,question_of_interest).sort_index())
        df_2020 = pd.DataFrame(count_then_return_percent(responses_df_2020,question_of_interest).sort_index())
        df_2019 = pd.DataFrame(count_then_return_percent(responses_df_2019,question_of_interest).sort_index())
        df_2018 = pd.DataFrame(count_then_return_percent(responses_df_2018,question_of_interest).sort_index())
    
        add_year_column_to_dataframes(
            df_2022,
            df_2021,
            df_2020,
            df_2019,
            df_2018)

        df_combined = pd.concat(
            [df_2018,
            df_2019,
            df_2020,
            df_2021,
            df_2022])

        df_combined[x_axis_title] = df_combined.index
        df_combined.columns = ['percentage','year',x_axis_title]
        return df_combined  
    
def combine_subset_of_data_from_multiple_years_for_multiple_choice_multiple_response_questions(question_of_interest,include_2017=None): 
    if include_2017 is None:
        question_of_interest = question_of_interest
        df_2022 = grab_subset_of_data(responses_df_2022, question_of_interest)
        df_2021 = grab_subset_of_data(responses_df_2021, question_of_interest)
        df_2020 = grab_subset_of_data(responses_df_2020, question_of_interest)
        df_2019 = grab_subset_of_data(responses_df_2019, question_of_interest)
        df_2018 = grab_subset_of_data(responses_df_2018, question_of_interest)
        
        add_year_column_to_dataframes(
            df_2022,
            df_2021,
            df_2020,
            df_2019,
            df_2018)

        df_combined = pd.concat(
            [df_2018,
            df_2019,
            df_2020,
            df_2021,
            df_2022])
        df_combined_counts = df_combined.groupby('year').count().reset_index()
        
        return df_combined, df_combined_counts
    
    else:
        question_of_interest = question_of_interest
        df_2022 = grab_subset_of_data(responses_df_2022, question_of_interest)
        df_2021 = grab_subset_of_data(responses_df_2021, question_of_interest)
        df_2020 = grab_subset_of_data(responses_df_2020, question_of_interest)
        df_2019 = grab_subset_of_data(responses_df_2019, question_of_interest)
        df_2018 = grab_subset_of_data(responses_df_2018, question_of_interest)
        df_2017 = grab_subset_of_data(responses_df_2017, question_of_interest)
       
        add_year_column_to_dataframes(
            df_2022,
            df_2021,
            df_2020,
            df_2019,
            df_2018,
            df_2017)

        df_combined = pd.concat(
            [df_2017,
            df_2018,
            df_2019,
            df_2020,
            df_2021,
            df_2022])
        df_combined_counts = df_combined.groupby('year').count().reset_index()

        return df_combined, df_combined_counts
    
def load_survey_data(base_dir,file_name,rows_to_skip=1):
    file_path = os.path.join(base_dir,file_name)
    df = pd.read_csv(file_path,low_memory=False,encoding='ISO-8859-1',skiprows=rows_to_skip)
    file_name = 'kaggle_survey_'+base_dir[-5:-1]+'.csv'
    files_present = glob.glob(file_name)
    if not files_present:
        df.to_csv(file_name,index=False)    
    return df
        

In [ ]:
directory = '/kaggle/working/individual_charts/'
if not os.path.exists(directory):
    os.mkdir(directory)
    
directory = '/kaggle/working/individual_charts/data/'
if not os.path.exists(directory):
    os.mkdir(directory)

directory = '/kaggle/working/individual_charts/charts/'
if not os.path.exists(directory):
    os.mkdir(directory)

base_dir_2017 = '/kaggle/input/kaggle-survey-2017/'
file_name_2017 = 'multipleChoiceResponses.csv'
responses_df_2017 = load_survey_data(base_dir_2017,file_name_2017,rows_to_skip=0)

base_dir_2018 = '/kaggle/input/kaggle-survey-2018/'
file_name_2018 = 'multipleChoiceResponses.csv'
responses_df_2018 = load_survey_data(base_dir_2018,file_name_2018)

base_dir_2019 = '/kaggle/input/kaggle-survey-2019/'
file_name_2019 = 'multiple_choice_responses.csv'
responses_df_2019 = load_survey_data(base_dir_2019,file_name_2019)

base_dir_2020 = '/kaggle/input/kaggle-survey-2020/'
file_name_2020 = 'kaggle_survey_2020_responses.csv'
responses_df_2020 = load_survey_data(base_dir_2020,file_name_2020)

base_dir_2021 = '/kaggle/input/kaggle-survey-2021/'
file_name_2021 = 'kaggle_survey_2021_responses.csv'
responses_df_2021 = load_survey_data(base_dir_2021,file_name_2021)

base_dir_2022 = '/kaggle/input/kaggle-survey-2022/'
file_name_2022 = 'kaggle_survey_2022_responses.csv'
responses_df_2022 = load_survey_data(base_dir_2022,file_name_2022)

In [ ]:
# Small repairs to 2022 data 
def replace_hyphen_with_en_dash(df):
    df.columns = df.columns.str.replace('Scikit-learn', 'Scikit—learn')
    df.columns = df.columns.str.replace('peer-reviewed', 'peer—reviewed')
    df.columns = df.columns.str.replace('Cloud-certification', 'Cloud—certification')
    df.columns = df.columns.str.replace('U-Net, Mask R-CNN', 'U—Net, Mask R—CNN')
    df.columns = df.columns.str.replace('Encoder-decoder', 'Encoder—decoder')
    df.columns = df.columns.str.replace('GPT-3', 'GPT—3')
    df.columns = df.columns.str.replace('gpt-3', 'gpt—3')
    df.columns = df.columns.str.replace('pre-trained', 'pre—trained')    
    df.columns = df.columns.str.replace('What-if', 'What—if')
    df.columns = df.columns.str.replace('Audit-AI', 'Audit—AI')
    return df
responses_df_2022 = replace_hyphen_with_en_dash(responses_df_2022)

In [ ]:
# # # Filter data according to whether or not the respondent gave specific answers (if you so decide)

# # # e.g.
# job_title = 'Data Scientist'
# # country = 'India'

# question_of_interest = 'Select the title most similar to your current role (or most recent title if retired): - Selected Choice'
# question_of_interest_2017 = 'CurrentJobTitleSelect'
# responses_df_2017 = responses_df_2017[responses_df_2017[question_of_interest_2017]==job_title] 
# responses_df_2018 = responses_df_2018[responses_df_2018[question_of_interest]==job_title] 
# responses_df_2019 = responses_df_2019[responses_df_2019[question_of_interest]==job_title] 
# responses_df_2020 = responses_df_2020[responses_df_2020[question_of_interest]==job_title] 
# responses_df_2021 = responses_df_2021[responses_df_2021[question_of_interest]==job_title] 
# responses_df_2022 = responses_df_2022[responses_df_2022[question_of_interest]==job_title]

# # question_of_interest = 'In which country do you currently reside?'
# # question_of_interest_2017 = 'Country'
# # responses_df_2017 = responses_df_2017[responses_df_2017[question_of_interest_2017]==country] 
# # responses_df_2018 = responses_df_2018[responses_df_2018[question_of_interest]==country] 
# # responses_df_2019 = responses_df_2019[responses_df_2019[question_of_interest]==country] 
# # responses_df_2020 = responses_df_2020[responses_df_2020[question_of_interest]==country] 
# # responses_df_2021 = responses_df_2021[responses_df_2021[question_of_interest]==country] 
# # responses_df_2022 = responses_df_2022[responses_df_2022[question_of_interest]==country] 

In [ ]:


question_of_interest = 'In which country do you currently reside?'
responses_df_2022.rename(columns={'United Kingdom of Great Britain and Northern Ireland':'United Kingdom (UK)'}, inplace=True)
responses_df_2022.replace(["United Kingdom of Great Britain and Northern Ireland"], 'United Kingdom (UK)',inplace=True)
responses_in_order = ['Other','India','United States of America','Brazil','Nigeria','Pakistan','Japan','China','Egypt','Indonesia','Mexico','Turkey','Russia']
responses_df_2022[question_of_interest][~responses_df_2022[question_of_interest].isin(responses_in_order)] = "Other"
percentages = count_then_return_percent(responses_df_2022,question_of_interest).sort_index()[responses_in_order]
percentages = percentages.sort_values(ascending=True)
title_for_chart = 'Most Common Nationalities Kaggle'
title_for_y_axis = '% of respondents'
orientation_for_chart = 'h'

bar_chart_multiple_choice(response_counts=percentages,
                 title=title_for_chart,
                 y_axis_title=title_for_y_axis,
                 orientation=orientation_for_chart,
                 num_choices=25) 
print("'Other' = any country not shown")

In [ ]:


question_name_alternate = 'Country'
responses_df_2022.rename(columns={'United Kingdom (UK)':'United Kingdom of Great Britain and Northern Ireland'}, inplace=True)
responses_df_2022.replace(["United Kingdom (UK)"], 'United Kingdom of Great Britain and Northern Ireland',inplace=True)
responses_df_2017[question_name_alternate].replace(['United States'], 'United States of America',inplace=True)
responses_df_2017[question_name_alternate].replace(["People 's Republic of China"], 'China',inplace=True)
responses_df_2017[question_name_alternate].replace(["United Kingdom"], 'United Kingdom of Great Britain and Northern Ireland',inplace=True)
responses_df_2017.rename(columns={question_name_alternate:'In which country do you currently reside?'}, inplace=True)
question_name_alternate = 'CurrentJobTitleSelect'
responses_df_2017.rename(columns={question_name_alternate:'Select the title most similar to your current role (or most recent title if retired): - Selected Choice'}, inplace=True)

subset_of_countries = ['Other','India','United States of America','Brazil','Nigeria','Pakistan','Japan','China','Egypt','Indonesia','Mexico','Turkey','Russia']
question_name = 'In which country do you currently reside?'
responses_df_2017[question_name][~responses_df_2017[question_name].isin(subset_of_countries)] = "Other"
responses_df_2018[question_name][~responses_df_2018[question_name].isin(subset_of_countries)] = "Other"
responses_df_2019[question_name][~responses_df_2019[question_name].isin(subset_of_countries)] = "Other"
responses_df_2020[question_name][~responses_df_2020[question_name].isin(subset_of_countries)] = "Other"
responses_df_2021[question_name][~responses_df_2021[question_name].isin(subset_of_countries)] = "Other"
responses_df_2022[question_name][~responses_df_2022[question_name].isin(subset_of_countries)] = "Other"

question_of_interest = 'In which country do you currently reside?'
column_of_interest = 'percentage'
title_for_chart = 'Most Common Nationalities on Kaggle (2017-2022)'
title_for_x_axis = ''
title_for_y_axis = '% of respondents'
country_df_combined = combine_subset_of_data_from_multiple_years(question_of_interest,title_for_x_axis,include_2017='yes')
country_df_combined = country_df_combined.sort_values(by=['year','percentage'], ascending=True)
country_df_combined.rename(columns={'United Kingdom of Great Britain and Northern Ireland':'United Kingdom (UK)'}, inplace=True)
country_df_combined.replace(["United Kingdom of Great Britain and Northern Ireland"], 'United Kingdom',inplace=True
                            
bar_chart_multiple_years(country_df_combined,
                                   title_for_x_axis,
                                   column_of_interest,
                                   title_for_chart,
                                   title_for_y_axis)
print("'Other' = any country not shown")                   

In [ ]:
question_of_interest = 'What is your age (# years)?'
percentages = count_then_return_percent(responses_df_2022,question_of_interest).sort_index()

title_for_chart = 'Age Distributions on Kaggle'
title_for_y_axis = '% of respondents'
bar_chart_multiple_choice(response_counts=percentages,
                 title=title_for_chart,
                 y_axis_title=title_for_y_axis,
                 orientation=orientation_for_chart)

In [ ]:
question_of_interest = 'What is your age (# years)?'
column_of_interest = 'percentage'
title_for_chart = 'Age distributions on Kaggle (2018-2022)'
title_for_x_axis = ''
title_for_y_axis = '% of respondents'
responses_df_2018[question_of_interest].replace(['70-79','80+'], '70+',inplace=True)
age_df_combined = combine_subset_of_data_from_multiple_years(question_of_interest,title_for_x_axis)

bar_chart_multiple_years(age_df_combined,
                                   title_for_x_axis,
                                   column_of_interest,
                                   title_for_chart,
                                   title_for_y_axis)


In [ ]:
question_of_interest = 'What is your gender? - Selected Choice'
responses_in_order = ['Man','Woman','Nonbinary','Prefer to self-describe','Prefer not to say']
percentages = count_then_return_percent(responses_df_2022,question_of_interest).sort_index()[responses_in_order].iloc[::-1]

title_for_chart = 'Gender Distributions on Kaggle'
title_for_y_axis = '% of respondents'
bar_chart_multiple_choice(response_counts=percentages,
                 title=title_for_chart,
                 y_axis_title=title_for_y_axis,
                 orientation=orientation_for_chart)

In [ ]:
responses_df_2017['GenderSelect'].replace(['Male'], 'Man',inplace=True)
responses_df_2017['GenderSelect'].replace(['Female'], 'Woman',inplace=True)
responses_df_2017['GenderSelect'].replace(['A different identity','Non-binary, genderqueer, or gender non-conforming'], 'Prefer to self-describe',inplace=True)
responses_df_2017['GenderSelect'].replace(['Non-binary, genderqueer, or gender non-conforming'], 'Nonbinary',inplace=True)
responses_df_2018['What is your gender? - Selected Choice'].replace(['Male'], 'Man',inplace=True)
responses_df_2018['What is your gender? - Selected Choice'].replace(['Female'], 'Woman',inplace=True)
responses_df_2019['What is your gender? - Selected Choice'].replace(['Male'], 'Man',inplace=True)
responses_df_2019['What is your gender? - Selected Choice'].replace(['Female'], 'Woman',inplace=True)

responses_df_2017['GenderSelect'].replace(['Nonbinary','Prefer not to say'], 'Prefer to self-describe',inplace=True)
responses_df_2017.columns = responses_df_2017.columns.str.replace('GenderSelect', 'What is your gender? - Selected Choice',regex=False)
responses_df_2018['What is your gender? - Selected Choice'].replace(['Nonbinary','Prefer not to say'], 'Prefer to self-describe',inplace=True)
responses_df_2019['What is your gender? - Selected Choice'].replace(['Nonbinary','Prefer not to say'], 'Prefer to self-describe',inplace=True)
responses_df_2020['What is your gender? - Selected Choice'].replace(['Nonbinary','Prefer not to say'], 'Prefer to self-describe',inplace=True)
responses_df_2021['What is your gender? - Selected Choice'].replace(['Nonbinary','Prefer not to say'], 'Prefer to self-describe',inplace=True)
responses_df_2022['What is your gender? - Selected Choice'].replace(['Nonbinary','Prefer not to say'], 'Prefer to self-describe',inplace=True)

question_of_interest = 'What is your gender? - Selected Choice'
column_of_interest = 'percentage'
title_for_chart = 'Gender distributions on Kaggle (2018-2022)'
title_for_x_axis = ''
title_for_y_axis = '% of respondents'

age_df_combined = combine_subset_of_data_from_multiple_years(question_of_interest,title_for_x_axis,include_2017='yes')
age_df_combined = age_df_combined.sort_values(by=['year','percentage'], ascending=True)

bar_chart_multiple_years(age_df_combined,
                                   title_for_x_axis,
                                   column_of_interest,
                                   title_for_chart,
                                   title_for_y_axis)

In [ ]:
question_of_interest = 'Are you currently a student? (high school, university, or graduate)'
percentages = count_then_return_percent(responses_df_2022,question_of_interest).sort_index()
title_for_chart = 'Students status for Kaggle Survey participants'
title_for_y_axis = '% of respondents'
  
bar_chart_multiple_choice(response_counts=percentages,
                 title=title_for_chart,
                 y_axis_title=title_for_y_axis,
                 orientation=orientation_for_chart)

In [ ]:
question_of_interest = 'On which platforms have you begun or completed data science courses?'
title_for_chart = 'Most popular online educational platforms for data science'
online_learning_platforms_df_2022 = grab_subset_of_data(responses_df_2022, question_of_interest)
online_learning_platforms_df_2022.columns = online_learning_platforms_df_2022.columns.str.replace("(direct from AWS, Azure, GCP, or similar)",'',regex=False)
online_learning_platforms_df_2022.columns = online_learning_platforms_df_2022.columns.str.replace("(resulting in a university degree)",'',regex=False)
bar_chart_multiple_choice_multiple_selection(online_learning_platforms_df_2022,title_for_chart,orientation='h')


In [ ]:
question_of_interest = 'On which platforms have you begun or completed data science courses'
question_of_interest_alternate = 'On which online platforms have you begun or completed data science courses'
#question_of_interest_alternate = 'On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Kaggle Learn Courses Courses Courses Courses Courses Courses Courses Courses Courses'
responses_df_2018.columns = responses_df_2018.columns.str.replace(question_of_interest_alternate, question_of_interest)
responses_df_2018.replace(['Kaggle Learn'], 'Kaggle Learn Courses',inplace=True)
responses_df_2018.columns = responses_df_2018.columns.str.replace('Kaggle Learn', 'Kaggle Learn Courses',regex=False)
responses_df_2018.replace(['Fast.AI'], 'Fast.ai',inplace=True)
responses_df_2018.columns = responses_df_2018.columns.str.replace('Fast.AI', 'Fast.ai')
responses_df_2018.replace(['Online University Courses'], 'University Courses (resulting in a university degree)',inplace=True)
responses_df_2018.columns = responses_df_2018.columns.str.replace('Online University Courses', 'University Courses (resulting in a university degree)',regex=False)
responses_df_2019.replace(['Kaggle Courses (i.e. Kaggle Learn)'], 'Kaggle Learn Courses',inplace=True)
responses_df_2019.columns = responses_df_2019.columns.str.replace('Kaggle Courses (i.e. Kaggle Learn)', 'Kaggle Learn Courses',regex=False)
question_of_interest = 'On which platforms have you begun or completed data science courses?'
title_for_chart = 'Most popular learning platforms (2018-2022)'
title_for_y_axis = '% of respondents'

learning_platform_df_combined, learning_platform_df_combined_counts = combine_subset_of_data_from_multiple_years_for_multiple_choice_multiple_response_questions(question_of_interest)
learning_platform_df_combined_percentages = convert_df_of_counts_to_percentages(learning_platform_df_combined, learning_platform_df_combined_counts)
learning_platform_df_combined_percentages.columns = learning_platform_df_combined_percentages.columns.str.replace("(resulting in a university degree)",'',regex=False)
learning_platform_df_combined_percentages = learning_platform_df_combined_percentages.loc[:, 
        ['year','Coursera','University Courses ','Kaggle Learn Courses','Udemy','Udacity','DataCamp','edX','Fast.ai','None','Other']]

df = learning_platform_df_combined_percentages.melt(id_vars=['year'], 
                                                    value_vars=['Coursera', 
                                                                'University Courses ',
                                                                'Kaggle Learn Courses', 
                                                                'Udemy', 
                                                                'Udacity',
                                                                'DataCamp', 
                                                                'edX', 
                                                                'Fast.ai'])
df = df.sort_values(by=['year','value'], ascending=True)
df.columns = df.columns.str.replace('variable', '')
bar_chart_multiple_years(df,'','value',title_for_chart,title_for_y_axis)



In [ ]:
question_of_interest = 'What products or platforms did you find to be most helpful when you first started studying data science?'
title_for_chart = 'Most helpful platforms for data science education'
online_learning_platforms_df_2022 = grab_subset_of_data(responses_df_2022, question_of_interest)
bar_chart_multiple_choice_multiple_selection(online_learning_platforms_df_2022,title_for_chart,orientation='h')

In [ ]:
question_of_interest = "Which of the following integrated development environments (IDE's) do you use on a regular basis?"
ide_df_2022 = grab_subset_of_data(responses_df_2022, question_of_interest)
title_for_chart = "Most Popular IDE's"
bar_chart_multiple_choice_multiple_selection(ide_df_2022,title_for_chart,orientation='h')


In [ ]:
correct_phrasing = "Jupyter Notebook / JupyterLab"
incorrect_phrasing = 'Jupyter/IPython'
responses_df_2018.columns = responses_df_2018.columns.str.replace(incorrect_phrasing, correct_phrasing,regex=False)

question_of_interest = "Which of the following integrated development environments (IDE's) do you use on a regular basis?"
alternate_phrasing = "Which of the following integrated development environments (IDE's) have you used at work or school in the last 5 years?"
responses_df_2018.columns = responses_df_2018.columns.str.replace(alternate_phrasing, question_of_interest,regex=False)
title_for_chart = "Most Popular IDE's 2018-2022"
x_axis_title = 'Percentage of respondents'
ide_df_combined, ide_df_combined_counts = combine_subset_of_data_from_multiple_years_for_multiple_choice_multiple_response_questions(question_of_interest,x_axis_title)

ide_df_combined_2 = ide_df_combined.copy()
columns_to_combine = ['Jupyter Notebook','JupyterLab ','Jupyter (JupyterLab, Jupyter Notebooks, etc) ','Jupyter Notebook / JupyterLab']
ide_df_combined_2['Jupyter Notebook / JupyterLab_'] = (ide_df_combined_2[columns_to_combine].notna()).any(axis="columns")
ide_df_combined_2 = ide_df_combined_2.drop(columns=columns_to_combine)
ide_df_combined_2['Jupyter Notebook / JupyterLab_'].replace([True], "Jupyter Notebook / JupyterLab",inplace=True)
ide_df_combined_2['Jupyter Notebook / JupyterLab_'].replace([False], np.nan,inplace=True)
ide_df_combined_2.columns = ide_df_combined_2.columns.str.replace('Jupyter Notebook / JupyterLab_', 'Jupyter Notebook / JupyterLab',regex=False)

columns_to_combine = ['MATLAB','MATLAB ']
ide_df_combined_2['MATLAB_'] = (ide_df_combined_2[columns_to_combine].notna()).any(axis="columns")
ide_df_combined_2 = ide_df_combined_2.drop(columns=columns_to_combine)
ide_df_combined_2['MATLAB_'].replace([True], "MATLAB",inplace=True)
ide_df_combined_2['MATLAB_'].replace([False], np.nan,inplace=True)
ide_df_combined_2.columns = ide_df_combined_2.columns.str.replace('MATLAB_', 'MATLAB',regex=False)

columns_to_combine = ['RStudio','RStudio ']
ide_df_combined_2['RStudio_'] = (ide_df_combined_2[columns_to_combine].notna()).any(axis="columns")
ide_df_combined_2 = ide_df_combined_2.drop(columns=columns_to_combine)
ide_df_combined_2['RStudio_'].replace([True], "RStudio",inplace=True)
ide_df_combined_2['RStudio_'].replace([False], np.nan,inplace=True)
ide_df_combined_2.columns = ide_df_combined_2.columns.str.replace('RStudio_', 'RStudio',regex=False)

columns_to_combine = ['Visual Studio Code', 'Visual Studio','Visual Studio / Visual Studio Code ','Visual Studio ', 'Visual Studio Code (VSCode) ','Click to write Choice 13']
ide_df_combined_2['Visual Studio / Visual Studio Code (VSCode)'] = (ide_df_combined_2[columns_to_combine].notna()).any(axis="columns")
ide_df_combined_2 = ide_df_combined_2.drop(columns=columns_to_combine)
ide_df_combined_2['Visual Studio / Visual Studio Code (VSCode)'].replace([True], "Visual Studio / Visual Studio Code (VSCode)",inplace=True)
ide_df_combined_2['Visual Studio / Visual Studio Code (VSCode)'].replace([False], np.nan,inplace=True)

columns_to_combine = ['PyCharm ', 'PyCharm'] 
ide_df_combined_2['PyCharm_'] = (ide_df_combined_2[columns_to_combine].notna()).any(axis="columns")
ide_df_combined_2 = ide_df_combined_2.drop(columns=columns_to_combine)
ide_df_combined_2['PyCharm_'].replace([True], "PyCharm",inplace=True)
ide_df_combined_2['PyCharm_'].replace([False], np.nan,inplace=True)
ide_df_combined_2.columns = ide_df_combined_2.columns.str.replace('PyCharm_', 'PyCharm',regex=False)

ide_df_combined_counts_2 = ide_df_combined_2.groupby('year').count().reset_index()
ide_df_combined_percentages = convert_df_of_counts_to_percentages(ide_df_combined_2, ide_df_combined_counts_2)

ide_df_combined_percentages = ide_df_combined_percentages.loc[:, 
        ['year','Jupyter Notebook / JupyterLab','Visual Studio / Visual Studio Code (VSCode)','RStudio','PyCharm','MATLAB']]

df = ide_df_combined_percentages.melt(id_vars=['year'], 
                                           value_vars=['Jupyter Notebook / JupyterLab','Visual Studio / Visual Studio Code (VSCode)','RStudio','PyCharm','MATLAB'])
df = df.sort_values(by=['year','value'], ascending=True)
df = df.rename(columns={'variable': ''})
bar_chart_multiple_years(df,'','value',title_for_chart,title_for_y_axis)

In [ ]:
question_of_interest = 'For how many years have you used machine learning methods?'
responses_in_order = ['I do not use machine learning methods','Under 1 year','1-2 years','2-3 years','3-4 years','4-5 years','5-10 years','10-20 years']
percentages = count_then_return_percent(responses_df_2022,question_of_interest).sort_index()[responses_in_order]
title_for_chart = "ML Experience Levels on Kaggle"
title_for_y_axis = '% of respondents'

bar_chart_multiple_choice(response_counts=percentages,
                 title=title_for_chart,
                 y_axis_title=title_for_y_axis,
                 orientation=orientation_for_chart) 

In [ ]:
question_of_interest = 'For how many years have you used machine learning methods?'
responses_df_2018.rename(columns={"For how many years have you used machine learning methods (at work or in school)?":question_of_interest}, inplace=True) 
responses_df_2018[question_of_interest].replace(["< 1 year"], "Under 1 year",inplace=True) 
responses_df_2018[question_of_interest].replace(["10-15 years"], "10-20 years",inplace=True) 
responses_df_2018[question_of_interest].replace(["20+ years"], "10-20 years",inplace=True) 
responses_df_2018[question_of_interest].replace(["I have never studied machine learning but plan to learn in the future"], "I do not use machine learning methods",inplace=True) 
responses_df_2018[question_of_interest].replace(["I have never studied machine learning and I do not plan to"], "I do not use machine learning methods",inplace=True) 
responses_df_2019[question_of_interest].replace(["< 1 years"], "Under 1 year",inplace=True) 
responses_df_2019[question_of_interest].replace(["10-15 years"], "10-20 years",inplace=True) 
responses_df_2019[question_of_interest].replace(["20+ years"], "20 or more years",inplace=True) 

title_for_chart = "ML experience for Kagglers (2018-2022)"
title_for_y_axis = '% of respondents'
title_for_x_axis = ''
ml_exp_df_combined = combine_subset_of_data_from_multiple_years(question_of_interest,title_for_x_axis).sort_values(by=['year','percentage'], ascending=True)

bar_chart_multiple_years(ml_exp_df_combined,
                                   title_for_x_axis,
                                   column_of_interest,
                                   title_for_chart,
                                   title_for_y_axis)

In [ ]:
question_of_interest = 'Which of the following machine learning frameworks do you use on a regular basis?'
title_for_chart = "Most popular machine learning frameworks"
ml_frameworks_df_2022 = grab_subset_of_data(responses_df_2022, question_of_interest)
bar_chart_multiple_choice_multiple_selection(ml_frameworks_df_2022,title_for_chart,orientation='h')

In [ ]:
# Combine TensorFlow and Keras into an "either TensorFlow OR Keras" column
ml_frameworks_df_2022 = grab_subset_of_data(responses_df_2022, question_of_interest)
columns_to_combine = ['TensorFlow ','Keras ']
ml_frameworks_df_2022['TensorFlow/Keras'] = (ml_frameworks_df_2022[columns_to_combine].notna()).any(axis="columns")
ml_frameworks_df_2022 = ml_frameworks_df_2022.drop(columns=columns_to_combine)
ml_frameworks_df_2022['TensorFlow/Keras'].replace([True], "TensorFlow/Keras",inplace=True)
ml_frameworks_df_2022['TensorFlow/Keras'].replace([False], np.nan,inplace=True)
# Same with PyTorch and PyTorch Lightning
columns_to_combine = ['PyTorch ','PyTorch Lightning ','Fast.ai ']
ml_frameworks_df_2022['PyTorch/PyTorch Lightning/Fast.ai'] = (ml_frameworks_df_2022[columns_to_combine].notna()).any(axis="columns")
ml_frameworks_df_2022 = ml_frameworks_df_2022.drop(columns=columns_to_combine)
ml_frameworks_df_2022['PyTorch/PyTorch Lightning/Fast.ai'].replace([True], "PyTorch/PyTorch Lightning/Fast.ai",inplace=True)
ml_frameworks_df_2022['PyTorch/PyTorch Lightning/Fast.ai'].replace([False], np.nan,inplace=True)
# Same with Xgboost, LightGBM, and CatBoost
columns_to_combine = ['Xgboost ','LightGBM ','CatBoost ']
ml_frameworks_df_2022['Xgboost/LightGBM/CatBoost'] = (ml_frameworks_df_2022[columns_to_combine].notna()).any(axis="columns")
ml_frameworks_df_2022 = ml_frameworks_df_2022.drop(columns=columns_to_combine)
ml_frameworks_df_2022['Xgboost/LightGBM/CatBoost'].replace([True], "Xgboost/LightGBM/CatBoost",inplace=True)
ml_frameworks_df_2022['Xgboost/LightGBM/CatBoost'].replace([False], np.nan,inplace=True)

ml_frameworks_df_2022 = ml_frameworks_df_2022.loc[:, 
        ['Scikit—learn ','TensorFlow/Keras','PyTorch/PyTorch Lightning/Fast.ai','Xgboost/LightGBM/CatBoost']] 
title_for_chart = "Most popular consolidated machine learning frameworks"
bar_chart_multiple_choice_multiple_selection(ml_frameworks_df_2022,title_for_chart,orientation='h')

In [ ]:
question_of_interest = 'Do you use any of the following hosted notebook products?'
title_for_chart = "Most popular hosted notebooks in 2022"
notebooks_df_2022 = grab_subset_of_data(responses_df_2022, question_of_interest)
bar_chart_multiple_choice_multiple_selection(notebooks_df_2022,title_for_chart,orientation='h')

In [ ]:
question_of_interest_original = 'Which of the following hosted notebook products do you use on a regular basis?'
question_of_interest_new = 'Do you use any of the following hosted notebook products?'
responses_df_2021.columns = responses_df_2021.columns.str.replace(question_of_interest_original, question_of_interest_new,regex=False)
notebooks_df_2021 = grab_subset_of_data(responses_df_2021, question_of_interest_new)

question_of_interest_original = 'Which of the following hosted notebook products do you use on a regular basis?'
question_of_interest_new = 'Do you use any of the following hosted notebook products?'
responses_df_2020.columns = responses_df_2020.columns.str.replace(question_of_interest_original, question_of_interest_new,regex=False)
notebooks_df_2020 = grab_subset_of_data(responses_df_2020, question_of_interest_new)

colab_text_to_replace = 'Google Colab '
colab_new_text = 'Colab Notebooks'
colab_answer = 'Which of the following hosted notebook products do you use on a regular basis?  (Select all that apply) - Selected Choice -  Google Colab '
kaggle_text_to_replace = 'Kaggle Notebooks (Kernels) '
kaggle_new_text = 'Kaggle Notebooks'
kaggle_answer = 'Which of the following hosted notebook products do you use on a regular basis?  (Select all that apply) - Selected Choice -  Kaggle Notebooks (Kernels) '
responses_df_2019[colab_answer] = responses_df_2019[colab_answer].str.replace(colab_text_to_replace,colab_new_text,regex=False)
responses_df_2019.columns = responses_df_2019.columns.str.replace(colab_text_to_replace, colab_new_text,regex=False)
responses_df_2019[kaggle_answer] = responses_df_2019[kaggle_answer].str.replace(kaggle_text_to_replace,kaggle_new_text,regex=False)
responses_df_2019.columns = responses_df_2019.columns.str.replace(kaggle_text_to_replace, kaggle_new_text,regex=False)
question_of_interest_original = 'Which of the following hosted notebook products do you use on a regular basis?'
question_of_interest_new = 'Do you use any of the following hosted notebook products?'
responses_df_2019.columns = responses_df_2019.columns.str.replace(question_of_interest_original, question_of_interest_new,regex=False)
notebooks_df_2019 = grab_subset_of_data(responses_df_2019, question_of_interest_new)

colab_text_to_replace = 'Google Colab'
colab_new_text = 'Colab Notebooks'
colab_answer = 'Which of the following hosted notebooks have you used at work or school in the last 5 years? (Select all that apply) - Selected Choice - Google Colab'
kaggle_text_to_replace = 'Kaggle Kernels'
kaggle_new_text = 'Kaggle Notebooks'
kaggle_answer = 'Which of the following hosted notebooks have you used at work or school in the last 5 years? (Select all that apply) - Selected Choice - Kaggle Kernels'
responses_df_2018[colab_answer] = responses_df_2018[colab_answer].str.replace(colab_text_to_replace,colab_new_text,regex=False)
responses_df_2018.columns = responses_df_2018.columns.str.replace(colab_text_to_replace, colab_new_text,regex=False)
responses_df_2018[kaggle_answer] = responses_df_2018[kaggle_answer].str.replace(kaggle_text_to_replace,kaggle_new_text,regex=False)
responses_df_2018.columns = responses_df_2018.columns.str.replace(kaggle_text_to_replace, kaggle_new_text,regex=False)
question_of_interest_original = 'Which of the following hosted notebooks have you used at work or school in the last 5 years?'
question_of_interest_new = 'Do you use any of the following hosted notebook products?'
responses_df_2018.columns = responses_df_2018.columns.str.replace(question_of_interest_original, question_of_interest_new,regex=False)
notebooks_df_2018 = grab_subset_of_data(responses_df_2018, question_of_interest_new)

question_of_interest = 'Do you use any of the following hosted notebook products?'
notebooks_df_combined, notebooks_df_combined_counts = combine_subset_of_data_from_multiple_years_for_multiple_choice_multiple_response_questions(question_of_interest)
notebooks_df_combined_percentages = convert_df_of_counts_to_percentages(notebooks_df_combined, notebooks_df_combined_counts)
notebooks_df_combined_percentages = notebooks_df_combined_percentages.loc[:, 
        ['year','None','Kaggle Notebooks','Colab Notebooks']]

title_for_chart = "Most popular hosted notebooks products 2018-2022"
title_for_y_axis = '% of respondents'
df = notebooks_df_combined_percentages.melt(id_vars=['year'], 
                                           value_vars=['None','Kaggle Notebooks','Colab Notebooks'])
df = df.rename(columns={'variable': ''})
bar_chart_multiple_years(df,'','value',title_for_chart,title_for_y_axis)

In [ ]:
question_of_interest = 'Do you download pre—trained model weights from any of the following services?'
title_for_chart = "Most popular ML model repositories"
models_df_2022 = grab_subset_of_data(responses_df_2022, question_of_interest)
bar_chart_multiple_choice_multiple_selection(models_df_2022,title_for_chart,orientation='h')

In [ ]:
question_of_interest = 'Which of the following ML model hubs/repositories do you use most often? - Selected Choice'
responses_in_order = [' Kaggle datasets ',' Huggingface Models ','  TensorFlow Hub ',
                      ' PyTorch Hub ',' Timm ',' NVIDIA NGC models  ',' ONNX models ',' Jumpstart ']
percentages = count_then_return_percent(responses_df_2022,question_of_interest).sort_index()[responses_in_order].iloc[::-1]
title_for_chart = "Favorite ML model repository (for users of multiple repositories)"
title_for_y_axis = '% of respondents'
orientation_for_chart = 'h'

bar_chart_multiple_choice(response_counts=percentages,
                 title=title_for_chart,
                 y_axis_title=title_for_y_axis,
                 orientation=orientation_for_chart,
                 num_choices=9) 

In [ ]:
question_of_interest = 'Select the title most similar to your current role (or most recent title if retired): - Selected Choice'
percentages = count_then_return_percent(responses_df_2022,question_of_interest).sort_index()
percentages = percentages.sort_values(ascending=True)
title_for_chart = "Most common job titles on Kaggle"
title_for_y_axis = '% of respondents'

bar_chart_multiple_choice(response_counts=percentages,
                 title=title_for_chart,
                 y_axis_title=title_for_y_axis,
                 orientation=orientation_for_chart,
                 num_choices=15)

In [ ]:
question_of_interest = 'Select the title most similar to your current role (or most recent title if retired): - Selected Choice'
title_for_chart = "Most common job titles on Kaggle (2018-2022)"
title_for_y_axis = '% of respondents'
job_df_combined = combine_subset_of_data_from_multiple_years(question_of_interest,title_for_x_axis)
job_df_combined.replace(["Data Analyst (Business, Marketing, Financial, Quantitative, etc)"], 'Data Analyst',inplace=True)
job_df_combined = job_df_combined.rename({"Data Analyst (Business, Marketing, Financial, Quantitative, etc)": 'Data Analyst'})
job_df_combined = job_df_combined[job_df_combined.index.isin(['Data Analyst','Data Engineer','Data Scientist','Research Scientist','Software Engineer'])]

bar_chart_multiple_years(job_df_combined,
                                   title_for_x_axis,
                                   column_of_interest,
                                   title_for_chart,
                                   title_for_y_axis)

In [ ]:
question_of_interest = 'In what industry is your current employer/contract (or your most recent employer if retired)? - Selected Choice'
percentages = count_then_return_percent(responses_df_2022,question_of_interest).sort_index()
percentages = percentages.sort_values(ascending=True)
title_for_chart = 'Most common industries of employment on Kaggle'
title_for_y_axis = '% of respondents'
  
bar_chart_multiple_choice(response_counts=percentages,
                 title=title_for_chart,
                 y_axis_title=title_for_y_axis,
                 orientation=orientation_for_chart) 

In [ ]:
question_of_interest = 'Approximately how many individuals are responsible for data science workloads at your place of business?'
responses_in_order = ['0','1-2','3-4','5-9','10-14','15-19','20+']
percentages = count_then_return_percent(responses_df_2022,question_of_interest).sort_index()[responses_in_order]
title_for_chart = 'Most common data science team sizes'
title_for_y_axis = '% of respondents'
  
bar_chart_multiple_choice(response_counts=percentages,
                 title=title_for_chart,
                 y_axis_title=title_for_y_axis,
                 orientation=orientation_for_chart)

In [ ]:
question_of_interest = 'Who/what are your favorite media sources that report on data science topics?'
title_for_chart = "Most popular data science media sources"
media_df_2022 = grab_subset_of_data(responses_df_2022, question_of_interest)
bar_chart_multiple_choice_multiple_selection(media_df_2022,title_for_chart,orientation='h')

In [ ]:

fig = px.line(df, x="Survey Year", y="New users in the 12 months prior to September", title='Yearly Kaggle account registrations 2017-2022')
fig.show()

In [3]:

"""Total Number of Responses 2017:  16716
Total Number of Responses 2018:  23859
Total Number of Responses 2019:  19717
Total Number of Responses 2020:  20036
Total Number of Responses 2021:  25973
Total Number of Responses 2022:  23997

Note that survey response counts are influenced by many different factors
If discussing the size of the Kaggle community, consider using the daily updated data 
from the Kaggle Meta Kaggle dataset instead (see chart below)

In 2022 there were more than 10 million registered users on Kaggle:  10886721
    
Divide the population into interesting subgroups and identify interesting insights.

Do students have different preferences as compared to professionals?
Do GCP customers have different preferences as compared to AWS customers?
Which cloud computing platforms have seen the most growth in recent years?
Do salaries scale according to experience levels? What traits might predict having a very high salary?
Credits / Attribution:

The idea to use pattern matching to identify which columns are associated with which questions came from @siddhantsadangi's notebook "Your Country VS The World" (see function grab_subset_of_data() for more detail).
Most plotting functions were adapted from examples in the plotly documentation.
This notebook (and every other public notebook on Kaggle) was released under an Apache 2.0 license. Consider clicking on the "copy & edit" button in the top right corner and then you can focus on some subset of the community that you find to be interesting!
This notebook makes use of 
the following Python libraries:

numpy: 1.21.6
pandas: 1.3.5
seaborn: 0.11.2
plotly express: 0.4.1  """

'Total Number of Responses 2017:  16716\nTotal Number of Responses 2018:  23859\nTotal Number of Responses 2019:  19717\nTotal Number of Responses 2020:  20036\nTotal Number of Responses 2021:  25973\nTotal Number of Responses 2022:  23997\n\nNote that survey response counts are influenced by many different factors\nIf discussing the size of the Kaggle community, consider using the daily updated data \nfrom the Kaggle Meta Kaggle dataset instead (see chart below)\n\nIn 2022 there were more than 10 million registered users on Kaggle:  10886721\n    \nDivide the population into interesting subgroups and identify interesting insights.\n\nDo students have different preferences as compared to professionals?\nDo GCP customers have different preferences as compared to AWS customers?\nWhich cloud computing platforms have seen the most growth in recent years?\nDo salaries scale according to experience levels? What traits might predict having a very high salary?\nCredits / Attribution:\n\nThe id